In [1]:
import requests #Request
from bs4 import BeautifulSoup #Parse
import re # Split type of Pokemon: GrassPsycho ==> [Grass, Psycho]
import urllib # download
import shutil # download

In [2]:
r = requests.get('https://pokemondb.net/pokedex/all') 
soup = BeautifulSoup(r.text, 'html.parser') 

In [5]:
results = soup.find_all('tr')

In [64]:
records = []
URLs = []
for result in results:
    rows = result.find_all_next('td')
    URL = 'https://pokemondb.net' + result.find_next('a')['href']
    Id = rows[0].text
    Name = rows[1].text
    Type = re.findall('[A-Z][^A-Z]*', rows[2].text)
    Total = rows[3].text
    URLs.append(URL)
    records.append((Id,Name,Type,Total,URL))

In [157]:
import pandas as pd  


In [163]:
df = pd.DataFrame(records, columns=['Id', 'Name', 'Type', 'Total', 'URL']) 
df.drop(df.index[0], inplace=True)
df.to_csv('Pokemon.csv', index=False, encoding='utf-8') 

In [155]:
i = 0
for url in URLs:
    req  = requests.get(url)
    Bsoup = BeautifulSoup(req.text, 'html.parser')
    img = Bsoup.find_all('div',attrs={'class':'figure'})
    download = img[0].find('img')['src']
    image_req = requests.get(download, stream=True)
    if image_req.status_code == 200:
        with open(str(i)+".png", 'wb') as f:
            image_req.raw.decode_content = True
            shutil.copyfileobj(image_req.raw, f)
    i += 1